In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import date
import talib
from sklearn.linear_model import *
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sklearn.metrics import accuracy_score
from sktime.forecasting.model_selection import SlidingWindowSplitter
import warnings
warnings.filterwarnings('ignore')

params_df = pd.read_csv("params.csv")

if len(sys.argv) == 2:
    prog_name, task_str = sys.argv
    param_row = int(task_str)
else:
    print("len(sys.argv)=%d so trying first param" % len(sys.argv))
    param_row = 0

param_dict = dict(params_df.iloc[param_row, :])

dataset_name = param_dict["dataset_name"]
forecast_horizon = param_dict["fh"]
window_size = param_dict["window_size"]
algorithm = param_dict["algorithm"]
train_size = param_dict["train_size"]
timeperiod = param_dict["sma"]

slope_threshold = param_dict["slope_threshold"]
year = str(param_dict["year"])
step_length = param_dict["step_length"]

root_data_dir = "/projects/genomic-ml/da2343/ml_project_2/data" 
dataset_dict = {
    "EURUSD_H1" : f"{root_data_dir}/EURUSD_H1_2007_2023_SMA_{timeperiod}.csv",
}
dataset_path = dataset_dict[dataset_name]

def extract_df(path, year, train_size):
    df = pd.read_csv(path, index_col=0)
    # Convert the Date_Time column to datetime format
    df['Date_Time'] = pd.to_datetime(df['Date_Time'])
    # Filter the dataframe by the year condition
    prev_df = df[df['Date_Time'].dt.year < int(year)]
    prev_df = prev_df.tail(train_size)
    current_yr_df = df[df['Date_Time'].dt.year == int(year)]
    filtered_df = pd.concat([prev_df, current_yr_df])
    return filtered_df
    
df = extract_df(dataset_path, year, train_size)

In [ ]:


learner_dict = {
    "LinearRegression": LinearRegression(),
}

splitter = SlidingWindowSplitter(window_length=train_size, 
                                 fh=np.arange(1, forecast_horizon + 1), 
                                 step_length=step_length)

accuracy_df_list = []
mapes = []
orders = []
outcomes = []

y = df[['SMA']]
offset = y.index[0]


def forecast_isolate(j):
    outcome = None
    
    # get the train and test indices
    splitter_y = splitter.split(y)
    train_idx, test_idx = next(islice(splitter_y, j, None))

    # train_idx, test_idx = splitter.split(y)[j]
    train_idx = train_idx + offset
    test_idx = test_idx + offset
    
    # get the train and test data
    y_train = y.loc[train_idx]
    y_test = y.loc[test_idx]
    
    # create a forecaster object using LinearRegression and recursive strategy
    regressor = learner_dict[algorithm]
    forecaster = make_reduction(regressor, 
                                window_length=window_size, 
                                strategy="recursive")
    forecaster.fit(y_train)
    fh = ForecastingHorizon(y_test.index, is_relative=False)
    y_pred = forecaster.predict(fh)
    mape = mean_absolute_percentage_error(y_test, y_pred, symmetric=False)
    
    x = np.arange(1, len(y_pred) + 1)
    slope, intercept = np.polyfit(x, y_pred, 1)
    y_fit = np.polyval([slope, intercept], x)
    mse = mean_squared_error(y_pred, y_fit)
    mse_scaled = mse * 10e8
    slope_scaled = slope * 10e5
    
    if abs(slope_scaled) > slope_threshold:
        has_traded = False
        if slope > 0:
            for i in test_idx:
                # check if there is a buy MACD crossover
                if df.loc[i, "MACD_Crossover_Change"] > 0 and \
                    has_traded == False and \
                    df.loc[i, "Close"] > df.loc[i, "SMA"] and \
                    df.loc[i, "MACD"] < 0:
                    ask_price = df.loc[i, "Close"]
                    tp_price = ask_price + 0.0150
                    sl_price = ask_price - 0.0100
                    
                    order = {
                        "ask_price": ask_price,
                        "take_profit_price": tp_price, 
                        "stop_loss_price": sl_price, 
                        "position": "long",
                        "MACD" : df.loc[i, "MACD"],
                        "SMA" : df.loc[i, "SMA"],
                    }
                    orders.append(order)
                    has_traded = True
                
                if has_traded == True:
                    close_price = df.loc[i, "Close"]
                    if close_price >= tp_price:
                        outcome = 1
                        break
                    elif close_price <= sl_price:
                        outcome = 0
                        break
        else:
            for i in test_idx:
                if df.loc[i, "MACD_Crossover_Change"] < 0 and \
                    has_traded == False and \
                    df.loc[i, "Close"] < df.loc[i, "SMA"] and \
                    df.loc[i, "MACD"] > 0:    
                    ask_price = df.loc[i, "Close"]
                    tp_price = ask_price - 0.0150
                    sl_price = ask_price + 0.0100
                    
                    order = {
                        "ask_price": ask_price,
                        "take_profit_price": tp_price, 
                        "stop_loss_price": sl_price, 
                        "position": "short",
                    }
                    orders.append(order)
                    has_traded = True
                
                if has_traded == True:
                    close_price = df.loc[i, "Close"]
                    if close_price <= tp_price:
                        outcome = 1
                        break
                    elif close_price >= sl_price:
                        outcome = 0
                        break

    return outcome, mape

split_y = splitter.split(y)
# create a parallel object with n_jobs processors
parallel = Parallel(n_jobs=-1)
# apply the forecast function to a range of indices in parallel
results = parallel(delayed(forecast_isolate)(j) for j, _ in enumerate(split_y))
# results = parallel(delayed(forecast_isolate)(j) for j in range(5))

# unpack the results into separate lists
outcomes, mapes = zip(*results)
outcomes = [x for x in outcomes if x is not None]

outcomes_array = np.array(outcomes)
mapes_array = np.array(mapes)


# apply the forecast_isolate function to the split_y list in parallel
# results = parallel.apply_async(forecast_isolate, split_y)

# get the results as a list
# results = results.get()

accuracy_df = pd.DataFrame({
    'accuracy': accuracy_score([1] * len(outcomes), outcomes),
    'no_of_trades': len(outcomes),
    'no_of_wins': sum(outcomes),
    'no_of_losses': len(outcomes) - sum(outcomes),
    'slope_threshold': slope_threshold,
    'year': year,
    'step_length': step_length,
    "mean_mape": np.mean(mapes),
    "dataset_name": dataset_name,
    "algorithm": algorithm,
    "train_size": train_size,
    "fh": forecast_horizon,
    "window_size": window_size,
    'sma': timeperiod,
}, index=[0])


# Save dataframe as a csv to output directory
out_file = f"results/{param_row}.csv"
accuracy_df.to_csv(out_file, encoding='utf-8', index=False)
print("Done!!")


In [9]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import date
import talib
from sklearn.linear_model import *
from sktime.forecasting.base import ForecastingHorizon
# from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error, mean_squared_error
from sklearn.metrics import accuracy_score
from sktime.forecasting.model_selection import SlidingWindowSplitter
from joblib import Parallel, delayed
from itertools import islice
import json
import warnings


warnings.filterwarnings('ignore')

params_df = pd.read_csv("params.csv")

if len(sys.argv) == 2:
    prog_name, task_str = sys.argv
    param_row = int(task_str)
else:
    print("len(sys.argv)=%d so trying first param" % len(sys.argv))
    param_row = 0

param_dict = dict(params_df.iloc[param_row, :])

dataset_name = param_dict["dataset_name"]
forecast_horizon = param_dict["fh"]
window_size = param_dict["window_size"]
algorithm = param_dict["algorithm"]
train_size = param_dict["train_size"]
timeperiod = param_dict["sma"]

slope_threshold = param_dict["slope_threshold"]
year = str(param_dict["year"])
step_length = param_dict["step_length"]

root_data_dir = "/projects/genomic-ml/da2343/ml_project_2/data" 
dataset_dict = {
    "EURUSD_H1" : f"{root_data_dir}/EURUSD/EURUSD_H1_200702210000_202304242100_Update.csv",
    "USDJPY_H1" : f"{root_data_dir}/USDJPY/USDJPY_H1_200705290000_202307282300_Update.csv",
    "GBPUSD_H1" : f"{root_data_dir}/GBPUSD/GBPUSD_H1_200704170000_202307282300_Update.csv",
    "AUDUSD_H1" : f"{root_data_dir}/AUDUSD/AUDUSD_H1_200704170000_202307282300_Update.csv",
    "USDCAD_H1" : f"{root_data_dir}/USDCAD/USDCAD_H1_200705300000_202307282300_Update.csv",
    "USDCHF_H1" : f"{root_data_dir}/USDCHF/USDCHF_H1_200704170000_202307282300_Update.csv",
    # "NZDUSD_H1" : f"{root_data_dir}/NZDUSD/NZDUSD_H1_200704170000_202307282300_Update.csv",
    "EURJPY_H1" : f"{root_data_dir}/EURJPY/EURJPY_H1_200705300000_202307282300_Update.csv",
    "EURGBP_H1" : f"{root_data_dir}/EURGBP/EURGBP_H1_200703270000_202307282300_Update.csv",
}

dataset_path = dataset_dict[dataset_name]
# Load the config file
config_path = "/projects/genomic-ml/da2343/ml_project_2/settings/config.json"
with open(config_path) as f:
  config = json.load(f)
# Get the take_profit and stop_loss levels from the config file
tp = config["trading_settings"][dataset_name]["take_profit"]
sl = config["trading_settings"][dataset_name]["stop_loss"]

def extract_df(path, year, train_size):
    df = pd.read_csv(path, index_col=0)
    # Convert the Date_Time column to datetime format
    df['Date_Time'] = pd.to_datetime(df['Date_Time'])
    # Filter the dataframe by the year condition
    prev_df = df[df['Date_Time'].dt.year < int(year)]
    prev_df = prev_df.tail(train_size)
    current_yr_df = df[df['Date_Time'].dt.year == int(year)]
    filtered_df = pd.concat([prev_df, current_yr_df])
    return filtered_df
    
df = extract_df(dataset_path, year, train_size)

len(sys.argv)=11 so trying first param


In [12]:
y = df[['Close']]
offset = y.index[0]

In [28]:
df

,Open,High,Low,Close,Volume,Date_Time,SMA_20,SMA_30,SMA_50,SMA_100,...,STOCH_K,STOCH_D,WILLR,BBANDS_Upper,BBANDS_Middle,BBANDS_Lower,AD,ADOSC,VOLUME_RSI,MFI
6997,1.58280,1.58340,1.58150,1.58340,578,2008-04-11 10:00:00,1.576915,1.579793,1.579524,1.575482,...,82.011455,75.806796,-14.529915,1.586027,1.580340,1.574653,3939.870328,510.746403,49.766370,76.196830
6998,1.58360,1.58420,1.58000,1.58020,564,2008-04-11 11:00:00,1.576745,1.579657,1.579700,1.575622,...,68.374028,76.017892,-43.362832,1.585227,1.581120,1.577013,3429.584613,364.080140,49.540572,73.659575
6999,1.58030,1.58290,1.57910,1.58220,1072,2008-04-11 12:00:00,1.576915,1.579580,1.579898,1.575772,...,65.853982,72.079822,-25.663717,1.584163,1.582040,1.579917,4106.637245,486.407949,57.139639,62.032120
7000,1.58230,1.58550,1.58070,1.58280,1085,2008-04-11 13:00:00,1.577345,1.579510,1.580100,1.575934,...,57.633406,63.953805,-23.076923,1.584422,1.582240,1.580058,3971.012245,449.078829,57.316788,66.412594
7001,1.58260,1.58470,1.58130,1.58220,631,2008-04-11 14:00:00,1.577715,1.579537,1.580312,1.576066,...,57.638889,60.375426,-28.205128,1.584313,1.582160,1.580007,3674.071068,298.501194,49.605752,61.586541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30003,1.29590,1.29667,1.29513,1.29606,2051,2011-12-30 19:00:00,1.294725,1.294333,1.293527,1.299759,...,44.462607,59.772400,-39.808917,1.298652,1.296968,1.295284,-162334.084644,-2452.127905,40.567804,45.533499
30004,1.29608,1.29658,1.29293,1.29408,2578,2011-12-30 20:00:00,1.294672,1.294496,1.293537,1.299640,...,29.176894,44.050341,-60.828025,1.298666,1.296182,1.293698,-163287.591493,-2616.284500,44.918850,45.618243
30005,1.29409,1.29464,1.29361,1.29410,2492,2011-12-30 21:00:00,1.294630,1.294589,1.293528,1.299511,...,21.096445,31.578649,-60.615711,1.297909,1.295474,1.293039,-163408.562367,-2484.086158,44.348264,40.619813
30006,1.29411,1.29589,1.29368,1.29557,2276,2011-12-30 22:00:00,1.294658,1.294736,1.293561,1.299395,...,30.818650,27.030663,-45.010616,1.296894,1.295144,1.293394,-161791.675489,-1689.714997,42.875145,45.746325


In [49]:
trades = []

# loop through all rows in the dataframe
for index in range(len(df)):
    i = index + offset

    if len(trades) != 0:
        prev_trade = trades[-1]
        # check if the previous trade was a long trade
        if prev_trade["position"] == 1:
            # check if the current close price is greater than the take profit price
            if df.loc[i, "Close"] >= prev_trade["take_profit_price"]:
                trades[-1]["label"] = 1
            # check if the current close price is less than the stop loss price
            elif df.loc[i, "Close"] <= prev_trade["stop_loss_price"]:
                trades[-1]["label"] = 0
            else:
                continue
        else:
            # check if the current close price is less than the take profit price
            if df.loc[i, "Close"] <= prev_trade["take_profit_price"]:
                trades[-1]["label"] = 1
            # check if the current close price is greater than the stop loss price
            elif df.loc[i, "Close"] >= prev_trade["stop_loss_price"]:
                trades[-1]["label"] = 0
            else:
                continue


    if df.loc[i, "MACD_Crossover_Change"] > 0:
        ask_price = df.loc[i, "Close"]
        tp_price = ask_price + tp
        sl_price = ask_price - sl
        current_position = 1

        local_order = {
            "index": i,
            "ask_price": ask_price,
            "take_profit_price": tp_price,
            "stop_loss_price": sl_price,
            "position": current_position,
            f"SMA_{timeperiod}": df.loc[i, f"SMA_{timeperiod}"],
            "MACD": df.loc[i, "MACD"],
            "MACD_Signal": df.loc[i, "MACD_Signal"],
            "MACD_Hist": df.loc[i, "MACD_Hist"],
            "RSI": df.loc[i, "RSI"],
            "ATR": df.loc[i, "ATR"],
            "ADX": df.loc[i, "ADX"],
            "AROON_Oscillator": df.loc[i, "AROON_Oscillator"],
            "WILLR": df.loc[i, "WILLR"],
            "OBV": df.loc[i, "OBV"],
            "CCI": df.loc[i, "CCI"],
            "PSAR": df.loc[i, "PSAR"],
            "AD": df.loc[i, "AD"],
            "ADOSC": df.loc[i, "ADOSC"],
            "VOLUME_RSI": df.loc[i, "VOLUME_RSI"],
            "MFI": df.loc[i, "MFI"],
            "Date_Time": df.loc[i, "Date_Time"],
            "label": None,
        }
        trades.append(local_order)

    elif df.loc[i, "MACD_Crossover_Change"] < 0:   
        ask_price = df.loc[i, "Close"]  
        tp_price = ask_price - tp
        sl_price = ask_price + sl
        current_position = 0

        local_order = {
            "index": i,
            "ask_price": ask_price,
            "take_profit_price": tp_price,
            "stop_loss_price": sl_price,
            "position": current_position,
            f"SMA_{timeperiod}": df.loc[i, f"SMA_{timeperiod}"],
            "MACD": df.loc[i, "MACD"],
            "MACD_Signal": df.loc[i, "MACD_Signal"],
            "MACD_Hist": df.loc[i, "MACD_Hist"],
            "RSI": df.loc[i, "RSI"],
            "ATR": df.loc[i, "ATR"],
            "ADX": df.loc[i, "ADX"],
            "AROON_Oscillator": df.loc[i, "AROON_Oscillator"],
            "WILLR": df.loc[i, "WILLR"],
            "OBV": df.loc[i, "OBV"],
            "CCI": df.loc[i, "CCI"],
            "PSAR": df.loc[i, "PSAR"],
            "AD": df.loc[i, "AD"],
            "ADOSC": df.loc[i, "ADOSC"],
            "VOLUME_RSI": df.loc[i, "VOLUME_RSI"],
            "MFI": df.loc[i, "MFI"],
            "Date_Time": df.loc[i, "Date_Time"],
            "label": None,
        }
        trades.append(local_order)
        
trades_df = pd.DataFrame(trades)

In [50]:
trades_df

,index,ask_price,take_profit_price,stop_loss_price,position,SMA_200,MACD,MACD_Signal,MACD_Hist,RSI,...,WILLR,OBV,CCI,PSAR,AD,ADOSC,VOLUME_RSI,MFI,Date_Time,label
0,7010,1.57210,1.56460,1.57710,0,1.571303,-0.000023,0.000549,-0.000572,34.049425,...,-85.350318,43801.0,-390.005469,1.584105,2870.773504,-151.612024,44.561206,35.933196,2008-04-14 00:00:00,0
1,7019,1.57840,1.58590,1.57340,1,1.571830,-0.001411,-0.001563,0.000151,54.168554,...,-31.927711,45796.0,42.368134,1.567310,4196.738238,619.282585,60.522114,76.010307,2008-04-14 09:00:00,1
2,7082,1.59530,1.58780,1.60030,0,1.578249,0.003495,0.003591,-0.000096,67.227953,...,-20.491803,45302.0,3.191974,1.597720,7968.336539,330.301115,43.901587,64.888168,2008-04-17 00:00:00,1
3,7118,1.58420,1.57670,1.58920,0,1.581937,-0.000471,-0.000163,-0.000309,37.542260,...,-84.444444,42615.0,-231.855917,1.595600,6858.735668,-17.614910,63.003380,52.953750,2008-04-18 12:00:00,0
4,7156,1.59020,1.58270,1.59520,0,1.584208,0.001692,0.001749,-0.000057,55.395126,...,-60.563380,45703.0,-52.809854,1.594500,6618.950500,-17.327247,44.445455,46.386773,2008-04-22 03:00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,29762,1.30266,1.29516,1.30766,0,1.323087,0.000512,0.000522,-0.000011,52.909987,...,-41.439206,374867.0,-53.992557,1.298602,-155800.253402,491.220183,60.195233,37.486416,2011-12-16 09:00:00,0
687,29823,1.30811,1.30061,1.31311,0,1.312012,0.001729,0.001788,-0.000058,58.627314,...,-40.761750,381766.0,-5.490196,1.307130,-147745.307401,-330.440759,40.524641,58.102689,2011-12-20 22:00:00,1
688,29951,1.29845,1.29095,1.30345,0,1.305420,-0.000614,-0.000099,-0.000515,16.989374,...,-86.078253,392120.0,-440.924523,1.307910,-153050.206063,-2348.468203,66.493351,44.507161,2011-12-28 15:00:00,1
689,29972,1.28674,1.27924,1.29174,0,1.304303,-0.003015,-0.002987,-0.000028,20.234647,...,-97.557841,361351.0,-228.244125,1.294161,-163777.666813,-2342.930655,56.651720,33.173712,2011-12-29 12:00:00,0


In [34]:
# get outcomes from trades_df using the label column
outcomes = trades_df["label"].tolist()

accuracy_df = pd.DataFrame({
    'accuracy': accuracy_score([1] * len(outcomes), outcomes),
    'no_of_trades': len(outcomes),
    'no_of_wins': sum(outcomes),
    'no_of_losses': len(outcomes) - sum(outcomes),
}, index=[0])

    

In [39]:
from datetime import datetime

# Define the two datetime objects
date1 = datetime(2008, 6, 30, 10, 0, 0)
date2 = datetime(2008, 7, 2, 14, 0, 0)

# Calculate the duration by subtracting the datetime objects
duration = date2 - date1

# Calculate the total duration in hours
total_hours = duration.days * 24 + duration.seconds / 3600

# Print the duration in hours
print(f"Duration: {total_hours} hours")


Duration: 52.0 hours


In [ ]:


learner_dict = {
    "LinearRegression": LinearRegression(),
}

splitter = SlidingWindowSplitter(window_length=train_size, 
                                 fh=np.arange(1, forecast_horizon + 1), 
                                 step_length=step_length)

accuracy_df_list = []
mapes = []
orders = []
outcomes = []

y = df[[f'SMA_{timeperiod}']]
offset = y.index[0]


def forecast_isolate(j):
    # get the train and test indices
    splitter_y = splitter.split(y)
    train_idx, test_idx = next(islice(splitter_y, j, None))

    train_idx = train_idx + offset
    test_idx = test_idx + offset
    
    # get the train and test data
    y_train = y.loc[train_idx]
    y_test = y.loc[test_idx]

    mape = 0
    current_position = None
    outcome = None
    local_order = {"index": None}
    has_traded = False
    
    for i in test_idx:
        # check if there is a buy MACD crossover
        # if df.loc[i, "MACD_Crossover_Change"] > 0 and \
        #     has_traded == False and \
        #     df.loc[i, "Close"] > df.loc[i, f"SMA_{timeperiod}"] and \
        #     df.loc[i, "MACD"] < 0:
        # if df.loc[i, "AROON_Oscillator"] <= -100 and \
        if df.loc[i, "MACD_Crossover_Change"] > 0 and \
            has_traded == False:
            ask_price = df.loc[i, "Close"]
            tp_price = ask_price + tp
            sl_price = ask_price - sl
            current_position = 1
            
            # tp_price = ask_price - tp
            # sl_price = ask_price + sl
            # current_position = 0
            
            local_order = {
                "index": i,
                "ask_price": ask_price,
                "take_profit_price": tp_price, 
                "stop_loss_price": sl_price, 
                "position": current_position,
                f"SMA_{timeperiod}" : df.loc[i, f"SMA_{timeperiod}"],
                "MACD" : df.loc[i, "MACD"],
                'MACD_Signal' : df.loc[i, "MACD_Signal"],
                "MACD_Hist" : df.loc[i, "MACD_Hist"],
                "RSI" : df.loc[i, "RSI"],
                "ATR" : df.loc[i, "ATR"],
                "ADX" : df.loc[i, "ADX"],
                "AROON_Oscillator" : df.loc[i, "AROON_Oscillator"],
                "WILLR" : df.loc[i, "WILLR"],

                "OBV" : df.loc[i, "OBV"],
                "CCI" : df.loc[i, "CCI"],
                "PSAR" : df.loc[i, "PSAR"],
                "AD" : df.loc[i, "AD"],
                "ADOSC" : df.loc[i, "ADOSC"],
                "VOLUME_RSI" : df.loc[i, "VOLUME_RSI"],                
                "MFI" : df.loc[i, "MFI"],
                "Date_Time" : df.loc[i, "Date_Time"],
                "label": None,
            }
            has_traded = True
            
        # elif df.loc[i, "MACD_Crossover_Change"] < 0 and \
        #     has_traded == False and \
        #     df.loc[i, "Close"] < df.loc[i, f"SMA_{timeperiod}"] and \
        #     df.loc[i, "MACD"] > 0:    
        # elif df.loc[i, "AROON_Oscillator"] >= 100 and \
        elif df.loc[i, "MACD_Crossover_Change"] < 0 and \
            has_traded == False:    
            ask_price = df.loc[i, "Close"]
            tp_price = ask_price - tp
            sl_price = ask_price + sl
            current_position = 0
            
            local_order = {
                "index": i,
                "ask_price": ask_price,
                "take_profit_price": tp_price, 
                "stop_loss_price": sl_price, 
                "position": current_position,
                f"SMA_{timeperiod}" : df.loc[i, f"SMA_{timeperiod}"],
                "MACD" : df.loc[i, "MACD"],
                'MACD_Signal' : df.loc[i, "MACD_Signal"],
                "MACD_Hist" : df.loc[i, "MACD_Hist"],
                "RSI" : df.loc[i, "RSI"],
                "ATR" : df.loc[i, "ATR"],
                "ADX" : df.loc[i, "ADX"],
                "AROON_Oscillator" : df.loc[i, "AROON_Oscillator"],
                "WILLR" : df.loc[i, "WILLR"],
                "OBV" : df.loc[i, "OBV"],
                "CCI" : df.loc[i, "CCI"],
                "PSAR" : df.loc[i, "PSAR"],
                "AD" : df.loc[i, "AD"],
                "ADOSC" : df.loc[i, "ADOSC"],
                "VOLUME_RSI" : df.loc[i, "VOLUME_RSI"],                
                "MFI" : df.loc[i, "MFI"],
                "Date_Time" : df.loc[i, "Date_Time"],
                "label": None,
            }
            has_traded = True
        
        if has_traded == True:
            close_price = df.loc[i, "Close"]
            if current_position == 1:
                if close_price >= tp_price:
                    outcome = 1
                    local_order["label"] = outcome
                    break
                elif close_price <= sl_price:
                    outcome = 0
                    local_order["label"] = outcome
                    break
            else:
                if close_price <= tp_price:
                    outcome = 1
                    local_order["label"] = outcome
                    break
                elif close_price >= sl_price:
                    outcome = 0
                    local_order["label"] = outcome
                    break

    return outcome, mape, local_order

split_y = splitter.split(y)
# create a parallel object with n_jobs processors
parallel = Parallel(n_jobs=-1)
# apply the forecast function to a range of indices in parallel
results = parallel(delayed(forecast_isolate)(j) for j, _ in enumerate(split_y))

# unpack the results into separate lists
outcomes, mapes, orders = zip(*results)
orders = list(orders)
orders_df = pd.DataFrame.from_dict(orders)
# drop the rows where any of the values is None
orders_df = orders_df.dropna()
# remove duplicate rows
orders_df = orders_df.drop_duplicates()
# remove duplicate indices
orders_df = orders_df.drop_duplicates(subset=['index'])
orders_df["year"] = year
orders_df["dataset_name"] = dataset_name

outcomes = [x for x in outcomes if x is not None]
outcomes_array = np.array(outcomes)
mapes_array = np.array(mapes)

accuracy_df = pd.DataFrame({
    'accuracy': accuracy_score([1] * len(outcomes), outcomes),
    'no_of_trades': len(outcomes),
    'no_of_wins': sum(outcomes),
    'no_of_losses': len(outcomes) - sum(outcomes),
    'slope_threshold': slope_threshold,
    'year': year,
    'step_length': step_length,
    "mean_mape": np.mean(mapes),
    "dataset_name": dataset_name,
    "algorithm": algorithm,
    "train_size": train_size,
    "fh": forecast_horizon,
    "window_size": window_size,
    'sma': timeperiod,
}, index=[0])


# # Save dataframe as a csv to output directory
accuracy_df.to_csv(f"results/{param_row}.csv", encoding='utf-8', index=False)
orders_df.to_csv(f"orders/{param_row}.csv", encoding='utf-8', index=False)
print("Done!!")
